In [1]:
import pandas as pd
from tqdm import tqdm
import warnings

warnings.filterwarnings(action='ignore')
tqdm.pandas()

df = pd.read_pickle('../data/filter_v2.pkl')

# df['token'] = [(i,j) for i,j in zip(df['text'], df['lang'])]
# df['token'] = df['token'].progress_apply(extract_noun)
df

,doc_id,date,country,keyword,text,lang,token,trans
0,jp_nok_126_1975,1975-09-16,Japan,North Korea,また、北朝鮮と日本との関係についても、漸次、貿易、人物、文化等の交流を積み重ねて、相互の理...,ja,"[また, 北朝鮮, と, 日本, と, の, 関係, に, つい, て, も, 漸次, 貿易...","In addition, regarding the relationship betwee..."
1,jp_nok_281_1990,1990-09-08,Japan,North Korea,自由民主党の金丸元副総理を団長とする訪朝団にきていただけるなら喜んで歓迎するという、前々から...,ja,"[自由, 民主, 党, の, 金丸, 元, 副, 総理, を, 団長, と, する, 訪朝,...",I have received a signal from a long time ago ...
2,jp_nok_336_1994,1994-03-04,Japan,North Korea,また、北朝鮮との国交正常化の問題については、今後とも核兵器開発問題等における動向を慎重に見守...,ja,"[また, 北朝鮮, と, の, 国交, 正常, 化, の, 問題, に, つい, て, は,...","In addition, regarding the issue of normalizin..."
3,jp_nok_347_1994,1994-07-18,Japan,North Korea,朝鮮半島においては、北朝鮮の金日成主席が逝去されましたが、私は、今回の事態が朝鮮半島の平和...,ja,"[朝鮮, 半島, に, おい, て, は, 北朝鮮, の, 金, 日成, 主席, が, 逝去...","On the Korean Peninsula, North Korean Presiden..."
4,jp_nok_417_1999,1999-01-19,Japan,North Korea,一方、北朝鮮に関しましては、米国、韓国などと緊密に連携をとりながら、先般の弾道ミサイルの発...,ja,"[一方, 北朝鮮, に, 関し, まし, て, は, 米国, 韓国, など, と, 緊密, ...","On the other hand, regarding North Korea, we w..."
...,...,...,...,...,...,...,...,...
1132,us_mex_8663_2001,2001-04-22,U.S.A.,Mexico,Canada has benefited; Mexico has benefited; th...,en,"[Canada, benefited;, Mexico, benefited;, Unite...",Canada has benefited; Mexico has benefited; th...
1133,us_mex_8739_2001,2001-08-24,U.S.A.,Mexico,The history of the relationship between Mexico...,en,"[history, relationship, Mexico, United, States...",The history of the relationship between Mexico...
1134,us_mex_8878_2002,2002-03-23,U.S.A.,Mexico,America's border with Mexico is a region of tr...,en,"[America's, border, Mexico, region, tremendous...",America's border with Mexico is a region of tr...
1135,us_mex_8902_2002,2002-05-04,U.S.A.,Mexico,America's relationship with Mexico is built on...,en,"[America's, relationship, Mexico, build, commo...",America's relationship with Mexico is built on...


In [2]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

documents = [TaggedDocument(doc.lower(), [id]) for id, doc in zip(df['doc_id'], df['trans'].str.lower())]
model = Doc2Vec(documents, vector_size=300, window=5, min_count=1, workers=4)
# model.save('./model/doc2vec_sentiment.bin')

In [3]:
# df = df2

diff = pd.DataFrame()
for word in tqdm(df['keyword'].unique()):
    df_temp = df.query(f"keyword == '{word}'")[['doc_id','date']]

    sims = []
    for id in df_temp['doc_id']:
        topn = model.dv.most_similar(id, topn=sys.maxsize)
        # temp = [id1 for id1, score in topn if score >= 0.7 and id1[:2] != id[:2]]
        temp = [id1 for id1, score in topn if score >= 0.7]
        temp = [id1 for id1 in temp if id1 in df_temp['doc_id'].to_list()]
        sims.extend([(id, x) for x in temp])

    edge = pd.DataFrame(sims, columns=['id1','id2'])

    date_from=[]; date_to=[]
    for data in edge.itertuples():
        date_from.append(df.query("doc_id == @data.id1")['date'].to_list()[0])
        date_to.append(df.query("doc_id == @data.id2")['date'].to_list()[0])

    edge['date_from'] = date_from; edge['date_to'] = date_to

    v1 = []; date=[]; keyword=[]

    for data in edge.itertuples():
        date1 = data.date_from
        date2 = data.date_to
        if abs((date1 - date2).days) <= 30:
            if date1 > date2:
                v1.append((data.id2, data.id1)); date.append((date2, date1))
            else:
                v1.append((data.id1, data.id2)); date.append((date1, date2))

    edge = pd.DataFrame(v1, columns=['id_from','id_to'])
    dates = pd.DataFrame(date, columns=['date_from','date_to'])
    edge['keyword'] = [word] * len(edge)
    edge = pd.concat([edge, dates], axis=1)


    edge['country_from'] = [i[:2] for i in edge['id_from']]
    edge['country_to'] = [i[:2] for i in edge['id_to']]

    edge['country_from'] = edge['country_from'].replace({'ko':'Korea','us':'U.S.A.','jp':'Japan'})
    edge['country_to'] = edge['country_to'].replace({'ko':'Korea','us':'U.S.A.','jp':'Japan'})

    diff = pd.concat([diff, edge], axis=0)
    # full = pd.concat([full, edge], axis=0)

diff['diff'] = diff['country_from'] != diff['country_to']
diff
# full

100%|██████████| 6/6 [19:24<00:00, 194.09s/it]   


,id_from,id_to,keyword,date_from,date_to,country_from,country_to,diff
0,jp_nok_126_1975,ko_nok_2126_1975,North Korea,1975-09-16,1975-10-03,Japan,Korea,True
1,jp_nok_126_1975,ko_nok_2125_1975,North Korea,1975-09-16,1975-10-01,Japan,Korea,True
2,ko_nok_3198_1990,jp_nok_281_1990,North Korea,1990-08-24,1990-09-08,Korea,Japan,True
3,ko_nok_3197_1990,jp_nok_281_1990,North Korea,1990-08-15,1990-09-08,Korea,Japan,True
4,ko_nok_3200_1990,jp_nok_281_1990,North Korea,1990-09-06,1990-09-08,Korea,Japan,True
...,...,...,...,...,...,...,...,...
23,ko_mex_7095_2016,ko_mex_7096_2016,Mexico,2016-04-01,2016-04-02,Korea,Korea,False
24,us_mex_7570_1997,us_mex_7577_1997,Mexico,1997-05-03,1997-05-06,U.S.A.,U.S.A.,False
25,us_mex_7570_1997,us_mex_7577_1997,Mexico,1997-05-03,1997-05-06,U.S.A.,U.S.A.,False
0,us_pan_3278_1977,us_pan_3291_1977,Panama,1977-08-23,1977-09-16,U.S.A.,U.S.A.,False


In [4]:
diff[diff['diff']]['keyword'].value_counts()

North Korea    420
Missile         22
Recession       10
Name: keyword, dtype: int64

In [5]:
diff1 = diff[diff['diff']]
ids = list(set(diff1['id_from']) | set(diff1['id_to']))
len(ids)

244

In [6]:
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
from scipy.sparse.linalg import svds
import re

def dummy_fun(doc):
    return doc

vectorizer = CountVectorizer(lowercase=False, tokenizer=dummy_fun)
X = vectorizer.fit_transform(df['token'].to_list())

X = np.array(X.toarray(), dtype=np.float64)
u, s, vh = svds(X, k=300)

from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse

vh1 = sparse.csr_matrix(vh)

sim = cosine_similarity(vh1.T, vh1.T)

name = vectorizer.get_feature_names_out()

# for x in lexicon.keys():
#     lexicon[x] = repl.get(lexicon[x], lexicon[x])

In [7]:
import numpy as np 

sent_dict = pd.read_csv('../data/survey_lex.csv', sep=',', names=['word','score'], )
sent_dict = {k:v for k,v in zip(sent_dict['word'], sent_dict['score'])}

# name = model.wv.key_to_index
name2idx = {k:v for v,k in enumerate(name)}
new2idx = {k:name2idx[k] for k in sent_dict.keys() if k in name}
idx2sent = {new2idx[k]: sent_dict[k] for k in sent_dict.keys() if k in name}

score = np.zeros(len(name))
np.put(score, list(idx2sent.keys()), list(idx2sent.values()))

sim = sim * score
lexicon = {k:v for k,v in zip(name, sim.sum(axis=1))}
lexicon.update(sent_dict)

In [8]:
import numbers
from collections import Counter

# counts = dict(Counter([i for l in df['token'] for i in l]))

def sentiment(sent):
    # counts = dict(Counter([i for i in sent]))
    for x in sent:
        if x in lexicon.keys():
            # result = [lexicon.get(x, x) * counts.get(x, x) for x in sent]
            result = [lexicon.get(x, x) for x in sent]
            result = [x for x in result if isinstance(x, numbers.Number)]
            result = np.array(result)
            # return np.sum(result) / len(sent)
            return np.mean(result)
            # return np.sum(result) / sum(counts.values())
            # return result
        else:
            pass
    

df['score'] = df['token'].apply(sentiment)
df1 = df.query("doc_id.isin(@ids)")
df1

,doc_id,date,country,keyword,text,lang,token,trans,score
0,jp_nok_126_1975,1975-09-16,Japan,North Korea,また、北朝鮮と日本との関係についても、漸次、貿易、人物、文化等の交流を積み重ねて、相互の理...,ja,"[また, 北朝鮮, と, 日本, と, の, 関係, に, つい, て, も, 漸次, 貿易...","In addition, regarding the relationship betwee...",0.030267
1,jp_nok_281_1990,1990-09-08,Japan,North Korea,自由民主党の金丸元副総理を団長とする訪朝団にきていただけるなら喜んで歓迎するという、前々から...,ja,"[自由, 民主, 党, の, 金丸, 元, 副, 総理, を, 団長, と, する, 訪朝,...",I have received a signal from a long time ago ...,0.002349
2,jp_nok_336_1994,1994-03-04,Japan,North Korea,また、北朝鮮との国交正常化の問題については、今後とも核兵器開発問題等における動向を慎重に見守...,ja,"[また, 北朝鮮, と, の, 国交, 正常, 化, の, 問題, に, つい, て, は,...","In addition, regarding the issue of normalizin...",0.136630
3,jp_nok_347_1994,1994-07-18,Japan,North Korea,朝鮮半島においては、北朝鮮の金日成主席が逝去されましたが、私は、今回の事態が朝鮮半島の平和...,ja,"[朝鮮, 半島, に, おい, て, は, 北朝鮮, の, 金, 日成, 主席, が, 逝去...","On the Korean Peninsula, North Korean Presiden...",0.200693
4,jp_nok_417_1999,1999-01-19,Japan,North Korea,一方、北朝鮮に関しましては、米国、韓国などと緊密に連携をとりながら、先般の弾道ミサイルの発...,ja,"[一方, 北朝鮮, に, 関し, まし, て, は, 米国, 韓国, など, と, 緊密, ...","On the other hand, regarding North Korea, we w...",-0.531673
...,...,...,...,...,...,...,...,...,...
1090,ko_mis_7124_2016,2016-06-13,Korea,Missile,북한은 올해 초 4차 핵실험과 미사일 발사 등 도발에 이어 핵보유국을 주장하며 공공...,kr,"[북한, 은, 올해, 초, 4, 차, 핵실험, 과, 미사일, 발사, 등, 도발, 에...",Following provocations such as the fourth nucl...,-0.257190
1093,ko_mis_7145_2016,2016-08-29,Korea,Missile,북한은 올해 초 4차 핵실험에 이어 최근까지 SLBM을 비롯한 탄도 미사일 발사 시...,kr,"[북한, 은, 올해, 초, 4, 차, 핵실험, 에, 이어, 최근, 까지, SLBM,...",Following the fourth nuclear test earlier this...,-0.297686
1094,ko_mis_7157_2016,2016-09-30,Korea,Missile,북한이 각종 미사일을 연속적으로 발사하고 있는 지금 이 상황에 우리 국민과 장병들을...,kr,"[북한, 이, 각종, 미사일, 을, 연속, 적, 으로, 발사, 하, 고, 있, 는,...",In this situation where North Korea is continu...,-0.158009
1095,ko_mis_7158_2016,2016-10-04,Korea,Missile,북한이 핵과 미사일에 광적으로 집착할수록 국제적 고립과 경제적 어려움만이 가중될 뿐...,kr,"[북한, 이, 핵, 과, 미사일, 에, 광적, 으로, 집착, 할수록, 국제, 적, ...",North Korea&#39;s fanatical obsession with nuc...,-0.058107


In [9]:
diff1 = diff[diff['diff']]
diff1

,id_from,id_to,keyword,date_from,date_to,country_from,country_to,diff
0,jp_nok_126_1975,ko_nok_2126_1975,North Korea,1975-09-16,1975-10-03,Japan,Korea,True
1,jp_nok_126_1975,ko_nok_2125_1975,North Korea,1975-09-16,1975-10-01,Japan,Korea,True
2,ko_nok_3198_1990,jp_nok_281_1990,North Korea,1990-08-24,1990-09-08,Korea,Japan,True
3,ko_nok_3197_1990,jp_nok_281_1990,North Korea,1990-08-15,1990-09-08,Korea,Japan,True
4,ko_nok_3200_1990,jp_nok_281_1990,North Korea,1990-09-06,1990-09-08,Korea,Japan,True
...,...,...,...,...,...,...,...,...
37,ko_mis_7145_2016,jp_mis_812_2016,Missile,2016-08-29,2016-09-09,Korea,Japan,True
40,jp_mis_821_2016,ko_mis_7157_2016,Missile,2016-09-26,2016-09-30,Japan,Korea,True
41,jp_mis_812_2016,ko_mis_7158_2016,Missile,2016-09-09,2016-10-04,Japan,Korea,True
42,jp_mis_821_2016,ko_mis_7158_2016,Missile,2016-09-26,2016-10-04,Japan,Korea,True


In [17]:
tri = [i for i in nx.find_cliques(G) if len(i) == 3]
tri

[['ko_nok_7114_2016', 'jp_nok_790_2016', 'us_nok_11661_2016'],
 ['jp_nok_790_2016', 'us_nok_11661_2016', 'ko_nok_7119_2016'],
 ['jp_nok_790_2016', 'us_nok_11661_2016', 'ko_nok_7124_2016'],
 ['jp_nok_790_2016', 'us_nok_11661_2016', 'ko_nok_7129_2016'],
 ['jp_nok_336_1994', 'us_nok_6593_1994', 'ko_nok_3672_1994'],
 ['jp_nok_336_1994', 'us_nok_6593_1994', 'ko_nok_3674_1994']]

In [23]:
tri1 = [i for l in tri for i in l]
tri1 = list(set(tri1))

In [19]:
tria = nx.triangles(G)
sum(list(tria.values()))

18

In [26]:
diff

,id_from,id_to,keyword,date_from,date_to,country_from,country_to,diff
0,jp_nok_126_1975,ko_nok_2126_1975,North Korea,1975-09-16,1975-10-03,Japan,Korea,True
1,jp_nok_126_1975,ko_nok_2125_1975,North Korea,1975-09-16,1975-10-01,Japan,Korea,True
2,ko_nok_3198_1990,jp_nok_281_1990,North Korea,1990-08-24,1990-09-08,Korea,Japan,True
3,ko_nok_3197_1990,jp_nok_281_1990,North Korea,1990-08-15,1990-09-08,Korea,Japan,True
4,ko_nok_3200_1990,jp_nok_281_1990,North Korea,1990-09-06,1990-09-08,Korea,Japan,True
...,...,...,...,...,...,...,...,...
23,ko_mex_7095_2016,ko_mex_7096_2016,Mexico,2016-04-01,2016-04-02,Korea,Korea,False
24,us_mex_7570_1997,us_mex_7577_1997,Mexico,1997-05-03,1997-05-06,U.S.A.,U.S.A.,False
25,us_mex_7570_1997,us_mex_7577_1997,Mexico,1997-05-03,1997-05-06,U.S.A.,U.S.A.,False
0,us_pan_3278_1977,us_pan_3291_1977,Panama,1977-08-23,1977-09-16,U.S.A.,U.S.A.,False


In [31]:
tri1

['jp_nok_790_2016',
 'ko_nok_3674_1994',
 'ko_nok_7124_2016',
 'us_nok_11661_2016',
 'ko_nok_7119_2016',
 'ko_nok_7129_2016',
 'us_nok_6593_1994',
 'ko_nok_3672_1994',
 'jp_nok_336_1994',
 'ko_nok_7114_2016']

In [44]:
diff1 = diff.query("id_from.isin(@tri1) & id_to.isin(@tri1)")
diff1 = diff1[diff1['diff']]
diff1 = diff1.sort_values(by='date_from')

In [41]:
df2 = df1.query("doc_id.isin(@tri1)")
df2 = df2.sort_values(by='date')

In [46]:
plot = df2
plot['wrap'] = plot['text'].str.wrap(60)
plot['wrap'] = plot['wrap'].apply(lambda x: x.replace('\n', '<br>'))

plot['stance'] = ['Negative' if x <= -0.01 else 'Neutral' if -0.01 < x < 0.01  else 'Positive' for x in plot['score']]

pd.options.plotting.backend = 'plotly'
fig = plot.plot(x="date", y="country", kind="scatter", color='stance', facet_col='keyword', facet_col_wrap=2, hover_data=['doc_id','wrap','score'], color_discrete_map={"Negative": "red", "Neutral": "green", "Positive": "blue"},)

# diff = pd.read_pickle('sim.pkl')
# diff = diff.query("diff == True")

annotations = []
for i, word in enumerate(plot['keyword'].unique()):
    for row in diff1.query("keyword == @word").itertuples():
        annotations.append(dict(x=row.date_to, y=row.country_to, xref=f'x{i+1}', yref=f'y{i+1}',
                        ax=row.date_from, ay=row.country_from, axref=f'x{i+1}', ayref=f'y{i+1}',
                        showarrow=True, arrowhead=3, arrowsize=1, arrowwidth=1) )
fig.update_layout(annotations=annotations)


fig.update_traces(marker_size=10)
fig.update_layout(width=1200, height=600)
fig.update_yaxes(tickangle=0)
fig.update_xaxes(type='category',categoryorder='category ascending')
fig.show()

In [50]:
diff_nk = diff.query("keyword == 'North Korea'")
diff.country_from.value_counts()

Korea     2912
U.S.A.     219
Japan       93
Name: country_from, dtype: int64

In [12]:
plot = df1.query("keyword == 'North Korea'")
plot['wrap'] = plot['text'].str.wrap(60)
plot['wrap'] = plot['wrap'].apply(lambda x: x.replace('\n', '<br>'))

plot['stance'] = ['Negative' if x <= -0.01 else 'Neutral' if -0.01 < x < 0.01  else 'Positive' for x in plot['score']]

pd.options.plotting.backend = 'plotly'
fig = plot.plot(x="date", y="country", kind="scatter", color='stance', facet_col='keyword', facet_col_wrap=2, hover_data=['doc_id','wrap','score'], color_discrete_map={"Negative": "red", "Neutral": "green", "Positive": "blue"},)

# diff = pd.read_pickle('sim.pkl')
# diff = diff.query("diff == True")

annotations = []
for i, word in enumerate(plot['keyword'].unique()):
    for row in diff.query("keyword == @word").itertuples():
        annotations.append(dict(x=row.date_to, y=row.country_to, xref=f'x{i+1}', yref=f'y{i+1}',
                        ax=row.date_from, ay=row.country_from, axref=f'x{i+1}', ayref=f'y{i+1}',
                        showarrow=True, arrowhead=3, arrowsize=1, arrowwidth=1) )
fig.update_layout(annotations=annotations)


fig.update_traces(marker_size=10)
fig.update_layout(width=1200, height=600)
fig.update_yaxes(tickangle=0)

fig.show()

In [6]:
plot.to_excel("result.xlsx")

In [ ]:
import chart_studio
chart_studio.tools.set_credentials_file(username='HenryYoon', api_key='v9HGGkyQsMk1e5jAcJlp')

import chart_studio.plotly as py
py.plot(fig, filename = 'Interactions between Korea, U.S.A., and Japan (North Korea)', auto_open=True)

In [36]:
plot['date'] = plot['date'].dt.strftime('%Y-%m-%d')

diff1['date_from'] = diff1['date_from'].dt.strftime('%Y-%m-%d')
diff1['date_to'] = diff1['date_to'].dt.strftime('%Y-%m-%d')

C:\Users\yunihg-kelab\AppData\Local\Temp\ipykernel_14732\1808398956.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [6]:
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt

sns.set_style('darkgrid')

# total = pd.read_pickle('../data/sim.pkl')
diff1 = diff[diff['diff']]

pd.options.plotting.backend = 'plotly'

plot1 = pd.crosstab(diff.year, diff.keyword, margins=True)
plot1 = plot1.drop(['All'], axis=0)
plot2 = pd.crosstab(diff1.year, diff1.keyword, margins=True)
plot2 = plot2.drop(['All'], axis=0)

plot = plot2.div(plot1['All'], axis=0).fillna(0)
# plot
plot.plot()

,id_from,id_to,keyword,date_from,date_to,country_from,country_to,different
0,jp_nok_126_1975,ko_nok_2126_1975,North Korea,1975-09-16,1975-10-03,Japan,Korea,True
1,jp_nok_126_1975,ko_nok_2125_1975,North Korea,1975-09-16,1975-10-01,Japan,Korea,True
2,ko_nok_3200_1990,jp_nok_281_1990,North Korea,1990-09-06,1990-09-08,Korea,Japan,True
3,ko_nok_3198_1990,jp_nok_281_1990,North Korea,1990-08-24,1990-09-08,Korea,Japan,True
4,ko_nok_3197_1990,jp_nok_281_1990,North Korea,1990-08-15,1990-09-08,Korea,Japan,True
...,...,...,...,...,...,...,...,...
33,jp_mis_821_2016,ko_mis_7157_2016,Missile,2016-09-26,2016-09-30,Japan,Korea,True
35,jp_mis_821_2016,ko_mis_7158_2016,Missile,2016-09-26,2016-10-04,Japan,Korea,True
38,jp_mis_812_2016,ko_mis_7158_2016,Missile,2016-09-09,2016-10-04,Japan,Korea,True
39,jp_mis_821_2016,ko_mis_7162_2016,Missile,2016-09-26,2016-10-10,Japan,Korea,True
